In [95]:
import findspark
findspark.init()

from pyspark.sql import  SparkSession
from pyspark.sql.functions import lit, col, when

In [3]:
spark = SparkSession.builder.appName('preprocesdid')\
    .config('spark.master', 'local[4]')\
    .config('spark.executor.memory', '1g')\
    .config("spark.sql.shuffle.partitions", 1)\
    .config('spark.driver.memory','1g')\
    .getOrCreate()

In [13]:
import os
name = r'D:\Workflow\work\csdid_r\R\5g10t.csv'
# os.path.exists(name)
data = spark.read.csv(name, header=True)
data.show(5)

+---+-----------------+---+-------+------+----------------+-----+
|  G|                X| id|cluster|period|               Y|treat|
+---+-----------------+---+-------+------+----------------+-----+
|  2|0.214945979570717|  1|     37|     1|4.39776569093816|    1|
|  2|0.214945979570717|  1|     37|     2|7.66172212986391|    1|
|  2|0.214945979570717|  1|     37|     3|6.83282829318896|    1|
|  2|0.214945979570717|  1|     37|     4|7.07668186230603|    1|
|  2|0.214945979570717|  1|     37|     5|8.99244332186859|    1|
+---+-----------------+---+-------+------+----------------+-----+
only showing top 5 rows



In [67]:
yname, gname, idname, tname = 'Y', 'G', 'id', 'period'
control_group = ['nevertreated', 'notyettreated']
anticipation = 0
panel = True
allow_unbalanced_panel = True,
weights_name = None

clustervar = None
xfmla = None

In [72]:
n_pre = data.count()
columns = [idname, tname, yname, gname]

if clustervar is not None:
	columns += [clustervar]
if weights_name is not None:
	columns += [weights_name]
	data = data.withColumn('_w', data[weights_name])
else:
	columns += ['_w']
	data = data.withColumn('_w', lit(1))

def form_to_strings(fmla : str = 'y ~ x + 1'):
	y, x = fmla.split('~')
	x_var = x.strip().split('+')
	x_var = [x.strip() for x in x_var]
	x_var = ['_intercept' if x == '1' else x for x in x_var]
	return x_var
data = data.withColumn('_intercept', lit(1))

# xfmla = 'y ~ X + 1'
if xfmla is None:
	x_var = ['_intercept']
	x_cov = data[x_var]
	n_cov = 1
else:
	x_var = form_to_strings(fmla=xfmla)
	n_cov = len(x_var)

columns += x_var

data = data[columns].na.drop('all')
ndiff = n_pre - data.count()

if ndiff != 0: 
	print(f'Dropped, {ndiff}, rows from original data due to missing data')


In [94]:
tlist = data.orderBy(col(tname)).select(tname).distinct()
glist = data.orderBy(col(gname)).select(gname).distinct()